In [51]:
print("testing python")

testing python


In [52]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

## LOAD DATA

In [53]:


DATA_PATH = os.path.join("..\ExerciseClassifier-master\MP_640x480_data")   # Create a folder for data storage
actions = np.array(['abdAdd', 'flexEx', 'shoulderPlane', 'idle'])
no_sequences = 310   # number of sequences (30 videos worth of data)
sequence_length = 30  # video is of 30 frames in length

In [54]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'abdAdd': 0, 'flexEx': 1, 'shoulderPlane': 2, 'idle': 3}

In [55]:
res = np.load(os.path.join(DATA_PATH, actions[0], str(29), "{}.npy".format(1)))
res.shape

(126,)

In [56]:
sequences, labels = [], []
for action in actions: #the 4 different workouts
    for sequence in range(no_sequences): #runs through all the 310 files
        window = []
        for frame_num in range(sequence_length): #runs for all 30 files
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [57]:
x = np.array(sequences)
#x[0,0,:]
#actions
y = np.array(labels)
x.shape
#y.shape

(1240, 30, 126)

In [58]:
from sklearn.model_selection import train_test_split


In [59]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42) #splitting as training and testing

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42) # splitting the training into further training and validation

x_train.shape

len(y_train)

y_test_train=np.empty((1,4))
for i in range(len(y_train)):
    if y_train[i]==1:
        y_temp=np.array([[0,0,0,1]])
    elif y_train[i]==2:
        y_temp=np.array([[0,0,1,0]]) 
    elif y_train[i]==3:
        y_temp=np.array([[0,1,0,0]]) 
    elif y_train[i]==4:
        y_temp=np.array([[1,0,0,0]]) 
    y_test_train=np.append(y_test_train,y_temp,axis=0) 
y_test_train=y_test_train[0:-1]
y_train=y_test_train

In [60]:
y_train=y_train.astype(float)
y_train


array([[6.23042070e-307, 4.67296746e-307, 1.69121096e-306,
        1.33511562e-306],
       [0.00000000e+000, 1.00000000e+000, 0.00000000e+000,
        0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 1.00000000e+000,
        0.00000000e+000],
       ...,
       [0.00000000e+000, 1.00000000e+000, 0.00000000e+000,
        0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 1.00000000e+000,
        0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        1.00000000e+000]])

## Build dataset

In [61]:
class CustomDatasets:
    def __init__(self, data , targets):
        self.data= data
        self.targets= targets

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        current_sample = self.data[idx, :]
        current_target = self.targets[idx, :]
        return {
        "x": torch.tensor(current_sample, dtype=torch.float),
        "y": torch.tensor(current_target, dtype=torch.float),
        }


In [62]:
y_train.shape

(892, 4)

In [63]:
dataset = CustomDatasets(x_train, y_train)
train_loader= DataLoader(dataset= dataset, batch_size=8, num_workers=0)
for data in train_loader:
    print(data["x"])

tensor([[[0.5718, 0.3906, 0.9999,  ..., 0.7332, 0.4143, 0.7395],
         [0.5717, 0.3907, 0.9999,  ..., 0.7335, 0.4140, 0.7404],
         [0.5716, 0.3908, 0.9999,  ..., 0.7341, 0.4133, 0.7413],
         ...,
         [0.5740, 0.3895, 0.9999,  ..., 0.7365, 0.4197, 0.7432],
         [0.5745, 0.3896, 0.9999,  ..., 0.7353, 0.4203, 0.7418],
         [0.5749, 0.3897, 0.9999,  ..., 0.7370, 0.4202, 0.7438]],

        [[0.5649, 0.4016, 0.9998,  ..., 0.3701, 0.2855, 0.3694],
         [0.5653, 0.4016, 0.9998,  ..., 0.3656, 0.2675, 0.3654],
         [0.5655, 0.4016, 0.9998,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.5677, 0.4012, 0.9999,  ..., 0.4078, 0.2051, 0.4096],
         [0.5677, 0.4011, 0.9999,  ..., 0.4119, 0.2094, 0.4135],
         [0.5676, 0.4010, 0.9999,  ..., 0.4139, 0.2118, 0.4154]],

        [[0.5578, 0.5315, 0.9997,  ..., 0.5469, 0.2129, 0.5474],
         [0.5578, 0.5315, 0.9997,  ..., 0.5470, 0.2128, 0.5475],
         [0.5578, 0.5315, 0.9997,  ..., 0.5469, 0.2123, 0.

## Building Pytorch model

In [64]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF

In [65]:
input_size = 126
hidden_size = 64
num_layers = 3
num_classes = 4
learning_rate = 0.0001
device = "cuda" if torch.cuda.is_available() else "cpu"

In [66]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.input_size = input_size
        self.num_classes = num_classes
        self.lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first=True)
        self.fc1 = nn.Linear(self.hidden_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 4)
        self.activation = torch.nn.Softmax(dim=1)
        
    def forward(self, x):
        
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        # batch_size, seq_len, hidden_size
        # out (N. 28, 128)
        
        out = out[:,-1,:].squeeze()
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return self.activation(out)

In [67]:
model = LSTM(input_size, hidden_size, num_layers, num_classes).to(device="cuda")

In [68]:
model

LSTM(
  (lstm): LSTM(126, 64, num_layers=3, batch_first=True)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=4, bias=True)
  (activation): Softmax(dim=1)
)

In [69]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [70]:
for epoch in range (20):
       #prediction = forward pass
    for data in train_loader:
        x_train=data["x"].to(device="cuda")
        y_train= data["y"].to(device="cuda")
        y_pred= model(x_train)
        #print(y_train)
        #print(y_pred)
        
        #loss
        l=loss_fn(y_train,y_pred)
    
        #gradients=backward pass
        l.backward()

        #update weights
        optimizer.step()
    
        #zero gradients
        optimizer.zero_grad()

        if epoch%2==0:
            print(f'epoch {epoch+1} loss= {l.item():.4f}')


epoch 1 loss= 1.4947
epoch 1 loss= 1.5026
epoch 1 loss= 1.5244
epoch 1 loss= 1.5018
epoch 1 loss= 1.5144
epoch 1 loss= 1.4582
epoch 1 loss= 1.5238
epoch 1 loss= 1.5010
epoch 1 loss= 1.5142
epoch 1 loss= 1.5245
epoch 1 loss= 1.5359
epoch 1 loss= 1.5007
epoch 1 loss= 1.5125
epoch 1 loss= 1.5139
epoch 1 loss= 1.4787
epoch 1 loss= 1.5246
epoch 1 loss= 1.5136
epoch 1 loss= 1.5120
epoch 1 loss= 1.5134
epoch 1 loss= 1.5138
epoch 1 loss= 1.5349
epoch 1 loss= 1.5131
epoch 1 loss= 1.4898
epoch 1 loss= 1.5110
epoch 1 loss= 1.5227
epoch 1 loss= 1.5334
epoch 1 loss= 1.4788
epoch 1 loss= 1.5115
epoch 1 loss= 1.5105
epoch 1 loss= 1.4884
epoch 1 loss= 1.5008
epoch 1 loss= 1.4890
epoch 1 loss= 1.4674
epoch 1 loss= 1.4776
epoch 1 loss= 1.5094
epoch 1 loss= 1.4875
epoch 1 loss= 1.4776
epoch 1 loss= 1.5336
epoch 1 loss= 1.5317
epoch 1 loss= 1.5092
epoch 1 loss= 1.4650
epoch 1 loss= 1.4539
epoch 1 loss= 1.4991
epoch 1 loss= 1.5093
epoch 1 loss= 1.5087
epoch 1 loss= 1.4865
epoch 1 loss= 1.4534
epoch 1 loss=

In [71]:
y_test_val=np.empty((1,4))
for i in range(len(y_val)):
    if y_val[i]==1:
        y_temp=np.array([[0,0,0,1]])
    elif y_val[i]==2:
        y_temp=np.array([[0,0,1,0]]) 
    elif y_val[i]==3:
        y_temp=np.array([[0,1,0,0]]) 
    elif y_val[i]==4:
        y_temp=np.array([[1,0,0,0]]) 
    y_test_val=np.append(y_test_val,y_temp,axis=0) 
y_test_val=y_test_val[0:-1]
y_val=y_test_val
y_val.shape


(224, 4)

In [72]:
x_val.shape
x_val_t= torch.from_numpy(x_val)

acc= y_predicted_cls.eq(y_val).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

IndentationError: unexpected indent (3030713838.py, line 5)